## Performance Tests of Coadd Catalog Access

Test the performance of data manipulations of the static coadd catalogs.

1. Identify trivial, moderate, and worst-case use case examples.
2. Measure performance on
    a. single patch
    b. a single tract
    c. the full dataset
    3. Record data sizes of each of the above a, b, c.
4. Determine if performance considerations mean we should generate a static file that contains a restricted set in columns.
5. Look into again using full tables functionality to write HDF5 files so that they can be read by column efficiently. This was previously not possible because of an error trying to write the thousands of columns in our full coadd catalogs. This is #158


In [1]:
import os

import numpy as np

import GCRCatalogs

If you want to use the GCR reader outside of NERSC environment, you can override the `base_dir`.

In [2]:
config = {}

trim_config = config.copy()
trim_config['filename_pattern'] = r'trim_merged_tract_\d+\.hdf5'
table_trim_config = config.copy()
table_trim_config['filename_pattern'] = r'table_trim_merged_tract_\d+\.hdf5'

trim_onetract_config = config.copy()
trim_onetract_config['filename_pattern'] = 'trim_merged_tract_4850.hdf5'
table_trim_onetract_config = config.copy()
table_trim_onetract_config['filename_pattern'] = 'table_trim_merged_tract_4850.hdf5'

In [3]:
%%timeit
gc_onetract = GCRCatalogs.load_catalog('dc2_coadd_run1.1p_tract4850', config)

397 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The trim files are 1/10 of the size of the full files.  The `load_catalog` doesn't load the data, but does need to open and touch each file to read the metadata.  This is only about 4 times faster for the trim catalogs.

In [4]:
%%timeit
gc_onetract_trim = GCRCatalogs.load_catalog('dc2_coadd_run1.1p_tract4850', trim_onetract_config)

98.3 ms ± 583 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
gc_onetract = GCRCatalogs.load_catalog('dc2_coadd_run1.1p_tract4850', config)
gc_onetract_trim = GCRCatalogs.load_catalog('dc2_coadd_run1.1p_tract4850', trim_onetract_config)

In [6]:
# This won't work until GCRCatalogs is updated to read the `table` formatted HDF5 files.
# gc_onetract_table_trim = GCRCatalogs.load_catalog('dc2_coadd_run1.1p_tract4850', table_trim_onetract_config)

In [7]:
%%timeit
gc = GCRCatalogs.load_catalog('dc2_coadd_run1.1p', config)

4.92 s ± 140 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
gc_trim = GCRCatalogs.load_catalog('dc2_coadd_run1.1p', trim_config)

1.29 s ± 36.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
gc = GCRCatalogs.load_catalog('dc2_coadd_run1.1p', config)
gc_trim = GCRCatalogs.load_catalog('dc2_coadd_run1.1p', trim_config)

Loading the GCR Catalog is, in principle, just the initialization of the catalog.  In practice the GCRCatalog reader does need to read through all of the metadata in the HDF5 files to figure out what's in there and available.  The onetract version is reading a 7.4 GB file that should fit in memory.  The full Run 1.1p is 78 GB, which does not fit in the average desktop memory.  This size could pontentially fit in the memory of various high-memory shared nodes.  This difference in size is conveniently roughly a factor of 10.  We should expect 

We can control the memory caching within GCR to clear the cache to reset for performance tests.  It's harder to control the underlying caching of the GPFS and kernel filesystem memory.

In [10]:
def compute_mean_color_slow(catalog):
    """Compute the mean g-r color of all objects in the 'catalog'.
    
    This is a trivial performance case.
    This isn't particularly immediately interesting, but it's a simple arithmetic operation between two columns.
    """
    average_gmr = np.mean(catalog['mag_g'] - catalog['mag_r'])
    return average_gmr

In [11]:
def compute_mean_color_faster(catalog):
    """Compute the mean g-r color of all objects in the 'catalog'.
    
    This is a trivial performance case.
    This isn't particularly immediately interesting, but it's a simple arithmetic operation between two columns.
    """
    data = catalog.get_quantities(['mag_g', 'mag_r'])
    average_gmr = np.mean(data['mag_g'] - data['mag_r'])
    return average_gmr

In [12]:
def compute_mean_color_faster_iter(catalog):
    """Compute the mean g-r color of all objects in the 'catalog' using iterator.
    
    This is a trivial performance case.
    This isn't particularly immediately interesting, but it's a simple arithmetic operation between two columns.
    """
    sum_gmr = count = 0
    for data in catalog.get_quantities(['mag_g', 'mag_r'], return_iterator=True):
        sum_gmr += np.sum(data['mag_g'] - data['mag_r'])
        count += len(data['mag_g'])
    return sum_gmr / count

In [13]:
# def compute_stellar_locus():

The average color calculation is 5 times faster with the trim files for one tract, using the slowest most naive way to access the quantities.

In [14]:
%%timeit
gc_onetract.clear_cache()
compute_mean_color_slow(gc_onetract)

4.66 s ± 985 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
gc_onetract_trim.clear_cache()
compute_mean_color_slow(gc_onetract_trim)

826 ms ± 40.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
gc.clear_cache()
compute_mean_color_slow(gc)

3min 6s ± 660 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
gc_onetract.clear_cache()
compute_mean_color_faster(gc_onetract)

4.31 s ± 1.11 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
gc_onetract.clear_cache()
compute_mean_color_faster_iter(gc_onetract)

3.64 s ± 91.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
gc.clear_cache()
compute_mean_color_faster(gc)

3min 43s ± 43 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
gc.clear_cache()
compute_mean_color_faster_iter(gc)

3min 12s ± 1.58 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%%timeit
gc_trim.clear_cache()
compute_mean_color_faster(gc_trim)

12.2 s ± 2.34 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
gc_trim.clear_cache()
compute_mean_color_faster_iter(gc_trim)

## Compare to using Pandas to read the data files directly.

In [21]:
import os
import pandas as pd

tract = 4850
datafile_basename = 'merged_tract_%d.hdf5' % tract
datafile_basename_trim = 'trim_' + datafile_basename
datafile_basename_table_trim = 'table_trim_' + datafile_basename

datafile = os.path.join(gc_trim.base_dir, datafile_basename)
datafile_trim = os.path.join(gc_trim.base_dir, datafile_basename_trim)
datafile_table_trim = os.path.join(gc_trim.base_dir, datafile_basename_table_trim)

key_prefix = 'coadd'
nx, ny = 8, 8
patches = ['%d%d' % (i, j) for i in range(nx) for j in range (ny)]  # Note '%d%d' instead of '%d,%d'
patch = patches[0]
key = '%s_%d_%s' % (key_prefix, tract, patch)

Reading just one patch of the tract.

In [22]:
%%timeit
df = pd.read_hdf(datafile, key=key)

40.5 ms ± 584 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
df_trim = pd.read_hdf(datafile_trim, key=key)

10.9 ms ± 345 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%%timeit
df_table_trim = pd.read_hdf(datafile_table_trim, key=key)

18.7 ms ± 416 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Now we'll load all of th patches in the tract

In [25]:
def load_tract_into_pandas(datafile, tract, key_prefix='coadd'):
    nx, ny = 8, 8
    patches = ['%d%d' % (i, j) for i in range(nx) for j in range (ny)]  # Note '%d%d' instead of '%d,%d'

    dfs = []
    for patch in patches:
        key = '%s_%d_%s' % (key_prefix, tract, patch)
        try:
            df = pd.read_hdf(datafile, key=key)
        except:
            continue
        dfs.append(df)

    df = pd.concat(dfs)
    return df

In [26]:
%%timeit
df_trim = load_tract_into_pandas(datafile_trim, tract=tract)

1.83 s ± 41.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
df_table_trim = load_tract_into_pandas(datafile_table_trim, tract=tract)

2.66 s ± 154 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Can we use Dask?

In [28]:
import dask as da
import dask.dataframe as dd

In [29]:
tract = 4850
datafile = os.path.join(gc_trim.base_dir, 'table_trim_merged_tract_%d.hdf5' % tract)
datafile_pattern = os.path.join(gc_trim.base_dir, 'table_trim_merged_tract_*.hdf5')

In [30]:
da_df = dd.read_hdf(datafile, key='coadd_*')
da_df_all = dd.read_hdf(datafile_pattern, key='coadd_*')

In [31]:
df2 = np.mean(da_df['g_mag'] - da_df['r_mag'])
df2_all = np.mean(da_df_all['g_mag'] - da_df_all['r_mag'])

In [32]:
%%timeit
df2.compute()

1.79 s ± 240 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%%timeit
df2_all.compute()

38.3 s ± 4.08 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Note that the full DASK calculation takes 20 times longer than the onetract calculation for a data volume 10-times larger.

DASK takes ~38 seconds to do the color average using the `table_trim_` file, while GCRCatalogs takes ~12 seconds using the `trim_` file.